In [1]:
import numpy as np
import pandas as pd
from collections import Counter
import math

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score
from sklearn import metrics
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.grid_search import GridSearchCV
from sklearn import svm
from sklearn.decomposition import RandomizedPCA
from sklearn.metrics import classification_report
from sklearn import preprocessing
import matplotlib.pyplot as plt
import scipy.optimize as opt
import glob
import os
import sys
import pickle
import time
import timeit

from sklearn.decomposition import RandomizedPCA

import theano
import theano.tensor as T
import lasagne

#np.random.seed(1337)  # for reproducibility
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten, Layer, Reshape, AutoEncoder, Merge
from keras.layers.convolutional import Convolution2D,MaxPooling2D,Convolution1D,MaxPooling1D,UpSampling2D, ZeroPadding2D
from keras.optimizers import SGD, Adam, RMSprop, Adadelta
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
import keras.utils.visualize_util as vutil
from keras.layers.recurrent import LSTM, GRU, SimpleRNN
from keras.regularizers import l1, l2, l1l2, activity_l2, activity_l1
import keras.layers.containers as containers
from keras.layers.noise import GaussianNoise
from keras.regularizers import ActivityRegularizer
from keras import backend as K
import keras.utils.visualize_util as vutil
from keras.models import model_from_json

import seaborn as sns

%matplotlib inline

from IPython.display import SVG

import h5py

np.set_printoptions(suppress=True)  #Supress exponent of the number 2e-2 == 0.02

%matplotlib inline

WARNING (theano.sandbox.cuda): CUDA is installed, but device gpu is not available  (error: Unable to get the number of gpus available: no CUDA-capable device is detected)
/usr/local/lib/python2.7/dist-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")
Using Theano backend.


In [2]:
#CPCG0184

In [3]:
#Read the data

GenomicFeatures_CPCG0184 = pd.read_csv(open('data/CPCG0184/GenomicFeatures_CPCG0184.txt','r'),sep='\t')
SNVCalls_CPCG0184    = pd.read_csv(open('data/CPCG0184/SNVCalls_CPCG0184.txt','r'),sep='\t') 

In [4]:
print GenomicFeatures_CPCG0184.shape
print SNVCalls_CPCG0184.shape

(128030, 15)
(128030, 111)


In [5]:
GenomicFeatures_CPCG0184.head()

,CHROM,POS,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias
0,chr1,13012,243,16,7230,286,176,9.0,39.5,CCA,2.697842,0.597015,NaN,39226,0.377622
1,chr1,13302,72,48,2913,138,88,15.0,65.0,ACG,2.453901,0.577114,intergenic,38936,0.557971
2,chr1,16288,170,23,5344,202,163,18.5,56.0,ACA,2.468966,0.492537,ncRNA_intronic,35950,0.524752
3,chr1,16957,164,14,5070,224,146,0.0,43.5,TCA,2.443662,0.601990,ncRNA_exonic,35281,0.522321
4,chr1,17408,230,22,6916,278,205,0.0,49.5,GCT,2.568493,0.621891,ncRNA_intronic,34830,0.600719


In [6]:
SNVCalls_CPCG0184.head()

,CHROM,POS,END,X2399004,X2642555,X2642557,X2673212,X2673226,X2677236,X2677261,X2677266,X2679050,X2679053,X2706569,X2711266,X2717532,X2717548,X2791041,X2811029,X2811031,
0,chr1,13011,13012,1,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,...
1,chr1,13301,13302,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
2,chr1,16287,16288,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
3,chr1,16956,16957,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
4,chr1,17407,17408,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...


In [7]:
np.unique(GenomicFeatures_CPCG0184['Trinucleotide'])

array(['ACA', 'ACC', 'ACG', 'ACT', 'ATA', 'ATC', 'ATG', 'ATT', 'CCA',
       'CCC', 'CCG', 'CCT', 'CNN', 'CTA', 'CTC', 'CTG', 'CTT', 'GCA',
       'GCC', 'GCG', 'GCT', 'GTA', 'GTC', 'GTG', 'GTT', 'NNG', 'TCA',
       'TCC', 'TCG', 'TCT', 'TTA', 'TTC', 'TTG', 'TTT'], 
      dtype='|S3')

In [8]:
CPCG0184 = pd.merge(GenomicFeatures_CPCG0184, SNVCalls_CPCG0184,left_on=['CHROM', 'POS'], right_on=['CHROM','END'])


In [9]:
CPCG0184.head() #182+15

,CHROM,POS_x,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias,POS_y,END,X2399004,X2642555,X2642557,
0,chr1,13012,243,16,7230,286,176,9.0,39.5,CCA,2.697842,0.597015,NaN,39226,0.377622,13011,13012,1,0,0,...
1,chr1,13302,72,48,2913,138,88,15.0,65.0,ACG,2.453901,0.577114,intergenic,38936,0.557971,13301,13302,0,0,0,...
2,chr1,16288,170,23,5344,202,163,18.5,56.0,ACA,2.468966,0.492537,ncRNA_intronic,35950,0.524752,16287,16288,1,0,0,...
3,chr1,16957,164,14,5070,224,146,0.0,43.5,TCA,2.443662,0.601990,ncRNA_exonic,35281,0.522321,16956,16957,0,0,0,...
4,chr1,17408,230,22,6916,278,205,0.0,49.5,GCT,2.568493,0.621891,ncRNA_intronic,34830,0.600719,17407,17408,1,0,0,...


In [10]:
CPCG0184_X = CPCG0184.iloc[:,2:15] #Exclude CHROM, POS
CPCG0184_Y = CPCG0184.iloc[:,17:]  #Exclude CHROM_y, POS_y, END

In [11]:
print CPCG0184_X.shape
print CPCG0184_Y.shape

(128030, 13)
(128030, 108)


In [12]:
CPCG0184_X[:2]

,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias
0,243,16,7230,286,176,9,39.5,CCA,2.697842,0.597015,NaN,39226,0.377622
1,72,48,2913,138,88,15,65.0,ACG,2.453901,0.577114,intergenic,38936,0.557971


In [13]:
CPCG0184_Y[:2]

,X2399004,X2642555,X2642557,X2673212,X2673226,X2677236,X2677261,X2677266,X2679050,X2679053,X2706569,X2711266,X2717532,X2717548,X2791041,X2811029,X2811031,X2811034,X2811037,X2811188,
0,1,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,...
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...


In [14]:
totalSample = CPCG0184_Y.shape[0]
CPCG0184_Y_Val = np.zeros((totalSample,1)) #Will be our label now

print totalSample
print CPCG0184_Y_Val.shape

128030
(128030, 1)


In [15]:
CPCG0184_Y.shape

(128030, 108)

In [16]:
CPCG0184_Y.values[0].shape

(108,)

In [17]:
for i in range(totalSample):
    CPCG0184_Y_Val[i] = np.argmax(np.bincount(CPCG0184_Y.values[i]))

In [18]:
CPCG0184_Y_Val_1 = pd.DataFrame(CPCG0184_Y_Val)
CPCG0184_Y_Val_1.columns = ['Label']

In [19]:
CPCG0184_Y_Val_1[:3]

,Label
0,0
1,0
2,0


In [20]:
CPCG0184_dataframe = pd.concat([CPCG0184_X, CPCG0184_Y_Val_1],axis=1)

In [21]:
CPCG0184_dataframe.shape

(128030, 14)

In [22]:
CPCG0184_dataframe.to_csv('CPCG0184.csv')

In [23]:
#Now we have variable importance ...lets do analysis

In [24]:
%load_ext rpy2.ipython

In [26]:
%%R
# Read in the data
CPCG0184Frame = read.csv("CPCG0184.csv")
str(CPCG0184Frame)

'data.frame':	128030 obs. of  15 variables:
 $ X              : int  0 1 2 3 4 5 6 7 8 9 ...
 $ RefAllele      : int  243 72 170 164 230 199 109 47 71 114 ...
 $ NonRefAllele   : int  16 48 23 14 22 6 22 25 27 20 ...
 $ BaseQual       : int  7230 2913 5344 5070 6916 5250 3821 1963 2833 3888 ...
 $ TumourCoverage : int  286 138 202 224 278 228 147 75 104 146 ...
 $ NormalCoverage : int  176 88 163 146 205 151 106 45 51 114 ...
 $ MapQual        : num  9 15 18.5 0 0 0 23 29 46 0 ...
 $ ReadPosition   : num  39.5 65 56 43.5 49.5 49 66 42 73 52 ...
 $ Trinucleotide  : Factor w/ 34 levels "ACA","ACC","ACG",..: 9 3 1 27 21 28 5 30 34 33 ...
 $ HomopolymerRate: num  2.7 2.45 2.47 2.44 2.57 ...
 $ GCcontent      : num  0.597 0.577 0.493 0.602 0.622 ...
 $ GenomicLocation: Factor w/ 15 levels "","downstream",..: 1 5 8 7 8 1 5 5 5 5 ...
 $ DistanceSNP    : int  39226 38936 35950 35281 34830 34508 1110 45 829 2868 ...
 $ StrandBias     : num  0.378 0.558 0.525 0.522 0.601 ...
 $ Label          : 

In [27]:
%%R
#sum(is.na(CPCG0183Frame$StrandBias))
sum(is.na(CPCG0184Frame$ReadPosition)) 

[1] 48


In [28]:
%%R
CPCG0184Frame$ReadPosition[is.na(CPCG0184Frame$ReadPosition)] = mean(CPCG0184Frame$ReadPosition, na.rm=TRUE)
CPCG0184Frame$StrandBias[is.na(CPCG0184Frame$StrandBias)] = mean(CPCG0184Frame$StrandBias, na.rm=TRUE)

print (sum(is.na(CPCG0184Frame$ReadPosition)) )
print (sum(is.na(CPCG0184Frame$StrandBias)) )

[1] 0
[1] 0


In [29]:
%%R
sum(is.na(CPCG0184Frame))  #No NA value now

[1] 0


In [30]:
%%R
write.csv(CPCG0184Frame, file = "CPCG0184_imputed.csv")  #Cleaned CSV without NA

In [31]:
%%R
library(h2o)
localH2O = h2o.init(ip = "localhost", port = 54321, startH2O = TRUE,min_mem_size = "3g")

/usr/local/lib/python2.7/dist-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: statmod

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/usr/local/lib/python2.7/dist-packages/rpy2/robjects/functions.py:106: UserWarning: 
----------------------------------------------------------------------

Your next step is to start H2O:
    > h2o.init()

For H2O package documentation, ask for help:
    > ??h2o

After starting H2O, you can use the Web UI at http://localhost:54321
For more information visit http://docs.h2o.ai

----------------------------------------------------------------------


  res = super(Function, self).__call__(*new_args, **new_kwargs)
/usr/local/lib/python2.7/dist-packages/rpy2/robjects/functions.py:106: UserWarning: 
Attaching package: ‘h2o’


  res = super(Function, self).__call__(*new_args, **new_kwargs)
/usr/local/lib/python2.7/dist-packages/rpy2/robjects/functions.py:106: UserWarning: The following objects are masked from ‘

 Connection successful!

R is connected to the H2O cluster: 
    H2O cluster uptime:         27 minutes 28 seconds 
    H2O cluster version:        3.9.1.3435 
    H2O cluster name:           H2O_started_from_R_rbhat_szm488 
    H2O cluster total nodes:    1 
    H2O cluster total memory:   26.58 GB 
    H2O cluster total cores:    56 
    H2O cluster allowed cores:  2 
    H2O cluster healthy:        TRUE 
    H2O Connection ip:          localhost 
    H2O Connection port:        54321 
    H2O Connection proxy:       NA 
    R Version:                  R version 3.3.0 beta (2016-03-30 r70404) 



In [32]:
%%R
## Import CSV as H2O:
csvPath = "/home/rbhat/Workspace/S3Lab_Projects/SMC-MetaChallange/CPCG0184_imputed.csv"
CPCG0184.hex          = h2o.importFile(path = csvPath, destination_frame = "CPCG0184.hex") 
train                 = as.data.frame(CPCG0184.hex)

#train  = subset(train,C1<20000)

train$Trinucleotide   = as.factor(train$Trinucleotide)
train$GenomicLocation = as.factor(train$GenomicLocation)
train$Label           = as.factor(train$Label)

train_h2o             = as.h2o(train)

  |======================================================================| 100%
  |======================================================================| 100%


In [33]:
%%R
#Training a deep learning model:---------------------------------------------------------#
model <- h2o.deeplearning( x = 3:15, # column numbers for predictors
                           y = 16,            # column number for label
                           training_frame = train_h2o,
                           activation = "RectifierWithDropout", 
                           input_dropout_ratio = 0.2,
                           hidden_dropout_ratios = c(0.5,0.5),
                           balance_classes = TRUE, 
                           hidden = c(10,8),  ## two hidden layers
                           epochs = 50,
                           variable_importances = T)

  |======================================================================| 100%


In [34]:
%%R
var_importance = h2o.varimp(model)
#print(var_importance)
h2o.varimp(model)

Variable Importances: 
        variable relative_importance scaled_importance percentage
1   NonRefAllele            1.000000          1.000000   0.308595
2 NormalCoverage            0.151707          0.151707   0.046816
3       BaseQual            0.148658          0.148658   0.045875
4      RefAllele            0.133894          0.133894   0.041319
5 TumourCoverage            0.111612          0.111612   0.034443

---
                     variable relative_importance scaled_importance percentage
56                  GCcontent            0.009834          0.009834   0.003035
57     GenomicLocation.exonic            0.007613          0.007613   0.002349
58          Trinucleotide.ATT            0.007398          0.007398   0.002283
59                 StrandBias            0.006075          0.006075   0.001875
60 GenomicLocation.intergenic            0.005877          0.005877   0.001814
61  Trinucleotide.missing(NA)            0.000000          0.000000   0.000000


In [35]:
%%R
write.table(var_importance, file="CPCG0184_variable_importance.csv")

In [36]:
var_importance = pd.read_csv(open('CPCG0184_variable_importance.csv','r'),sep=' ')
#pd.read_csv('CPCG0100_variable_importance.csv')

In [37]:
var_importance[:5]

,variable,relative_importance,scaled_importance,percentage
1,NonRefAllele,1.000000,1.000000,0.308595
2,NormalCoverage,0.151707,0.151707,0.046816
3,BaseQual,0.148658,0.148658,0.045875
4,RefAllele,0.133894,0.133894,0.041319
5,TumourCoverage,0.111612,0.111612,0.034443


In [38]:
GenomicFeatures_CPCG0184[:5]

,CHROM,POS,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias
0,chr1,13012,243,16,7230,286,176,9.0,39.5,CCA,2.697842,0.597015,NaN,39226,0.377622
1,chr1,13302,72,48,2913,138,88,15.0,65.0,ACG,2.453901,0.577114,intergenic,38936,0.557971
2,chr1,16288,170,23,5344,202,163,18.5,56.0,ACA,2.468966,0.492537,ncRNA_intronic,35950,0.524752
3,chr1,16957,164,14,5070,224,146,0.0,43.5,TCA,2.443662,0.601990,ncRNA_exonic,35281,0.522321
4,chr1,17408,230,22,6916,278,205,0.0,49.5,GCT,2.568493,0.621891,ncRNA_intronic,34830,0.600719


In [39]:
var_imp_dict = {}

for i in range(var_importance.shape[0]):
    var_imp_dict[var_importance.iloc[i,0]] = var_importance.iloc[i,1]

var_imp_dict

{'BaseQual': 0.148658066987991,
 'DistanceSNP': 0.039582252502441399,
 'GCcontent': 0.0098344003781676292,
 'GenomicLocation.UTR3': 0.050369914621114703,
 'GenomicLocation.UTR5': 0.058408152312040301,
 'GenomicLocation.downstream': 0.032754763960838297,
 'GenomicLocation.exonic': 0.0076126665808260406,
 'GenomicLocation.exonic;splicing': 0.056039232760667801,
 'GenomicLocation.intergenic': 0.0058770123869180705,
 'GenomicLocation.intronic': 0.010472397319972499,
 'GenomicLocation.missing(NA)': 0.017932562157511701,
 'GenomicLocation.ncRNA_UTR3': 0.038904733955860103,
 'GenomicLocation.ncRNA_exonic': 0.079050756990909604,
 'GenomicLocation.ncRNA_intronic': 0.022489931434392901,
 'GenomicLocation.ncRNA_splicing': 0.078374296426773099,
 'GenomicLocation.splicing': 0.0400108098983765,
 'GenomicLocation.upstream': 0.0144483223557472,
 'GenomicLocation.upstream;downstream': 0.066543936729431194,
 'HomopolymerRate': 0.022859552875161199,
 'MapQual': 0.085189744830131503,
 'NonRefAllele': 1.0,

In [40]:
noOfSample = GenomicFeatures_CPCG0184.shape[0]

CPCG0184_Dict = {}

for i in range(noOfSample):
    
    pos = GenomicFeatures_CPCG0184.iloc[i,1] #gets the pos value
    
    RefAllele      = GenomicFeatures_CPCG0184.iloc[i,2]*var_imp_dict['RefAllele']
    NonRefAllele   = GenomicFeatures_CPCG0184.iloc[i,3]*var_imp_dict['NonRefAllele']
    BaseQual       = GenomicFeatures_CPCG0184.iloc[i,4]*var_imp_dict['BaseQual']
    TumourCoverage = GenomicFeatures_CPCG0184.iloc[i,5]*var_imp_dict['TumourCoverage']
    NormalCoverage = GenomicFeatures_CPCG0184.iloc[i,6]*var_imp_dict['NormalCoverage']
    MapQual        = GenomicFeatures_CPCG0184.iloc[i,7]*var_imp_dict['MapQual']
    ReadPosition   = GenomicFeatures_CPCG0184.iloc[i,8]*var_imp_dict['ReadPosition']
    
    Trinucleotide  = 0
    if isinstance(GenomicFeatures_CPCG0184.iloc[i,9], str):
        Trinucleotide  = var_imp_dict['Trinucleotide.'+str(GenomicFeatures_CPCG0184.iloc[i,9])]
    HomopolymerRate= GenomicFeatures_CPCG0184.iloc[i,10]*var_imp_dict['HomopolymerRate']
    GCcontent      = GenomicFeatures_CPCG0184.iloc[i,11]*var_imp_dict['GCcontent']
    
    GenomicLocation = 0
    if isinstance(GenomicFeatures_CPCG0184.iloc[i,12], str):
        GenomicLocation= var_imp_dict['GenomicLocation.'+str(GenomicFeatures_CPCG0184.iloc[i,12])]
    DistanceSNP    = GenomicFeatures_CPCG0184.iloc[i,13]*var_imp_dict['DistanceSNP']
    StrandBias     = GenomicFeatures_CPCG0184.iloc[i,14]*var_imp_dict['StrandBias']
    
    #print 'RefAllele=',RefAllele,' NonRefAllele=',NonRefAllele,' BaseQual=',BaseQual,' TumourCoverage=',TumourCoverage
    #print 'NormalCoverage=',NormalCoverage,' MapQual=',MapQual,' ReadPosition=',ReadPosition,' Trinucleotide=',Trinucleotide
    #print 'HomopolymerRate=',HomopolymerRate,' GCcontent=',GCcontent,' GenomicLocation=',GenomicLocation,' DistanceSNP=',DistanceSNP
    #print 'StrandBias=',StrandBias
    CPCG0184_Dict[pos] = RefAllele+NonRefAllele+BaseQual+TumourCoverage+NormalCoverage+MapQual+ReadPosition+Trinucleotide+ \
                         HomopolymerRate+GCcontent+GenomicLocation+DistanceSNP+StrandBias

In [41]:
#Sorting dictionary
CPCG0184_Dict1 = [(k,v) for v,k in sorted([(v,k) for k,v in CPCG0184_Dict.items()],reverse=True)]

In [42]:
CPCG0184_Dict_Frame = pd.DataFrame(CPCG0184_Dict1, columns=['POS', 'Value'])

In [43]:
CPCG0184_Dict_Frame.to_csv("CPCG0184_Dict_Frame.csv")

In [23]:
CPCG0184_Dict_Frame = pd.read_csv("CPCG0184_Dict_Frame.csv")

In [24]:
CPCG0184_Dict_Frame[:10]

,Unnamed: 0,POS,Value
0,0,63742,104364.635490
1,1,282067,95738.000337
2,2,369223,92298.760862
3,3,393602,91334.798245
4,4,397722,91146.633246
5,5,590877,83513.715736
6,6,617679,82439.746316
7,7,638533,81710.718617
8,8,811178,74782.592182
9,9,42597791,65432.010300


In [25]:
GenomicFeatures_CPCG0184_1 = pd.concat([CPCG0184.iloc[:,0:16], CPCG0184_Y_Val_1],axis=1)

In [26]:
rows = CPCG0184_Dict_Frame['POS'][:10].values
print rows
GenomicFeatures_CPCG0184_Top10 = GenomicFeatures_CPCG0184_1.loc[GenomicFeatures_CPCG0184_1['POS_x'].isin(rows)]

[   63742   282067   369223   393602   397722   590877   617679   638533
   811178 42597791]


In [28]:
GenomicFeatures_CPCG0184_Top10[:10]

,CHROM,POS_x,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias,POS_y,Label
12856,chr10,42597791,5653,2360,236366,107911,97464,15,26.0,ATT,2.186207,0.363184,NaN,8727,0.535339,42597790,0
119895,chrX,63742,3,0,96,5,1,0,66.0,CCT,3.967213,0.701493,intergenic,2636226,0.800000,63741,0
119904,chrX,282067,9,4,143,26,13,0,24.5,TTT,5.511811,0.512438,downstream,2417901,0.576923,282066,0
119911,chrX,369223,8,3,225,17,11,0,71.0,CCG,1.955975,0.582090,intergenic,2330745,0.470588,369222,0
119912,chrX,393602,9,1,257,14,6,0,31.5,GCG,2.197368,0.646766,intergenic,2306366,0.571429,393601,0
119913,chrX,397722,3,1,96,6,2,0,89.5,ACA,1.262032,0.487562,intergenic,2302246,0.666667,397721,0
119918,chrX,590877,11,1,163,17,8,0,58.0,GCG,3.277778,0.676617,intronic,2109091,0.588235,590876,0
119919,chrX,617679,5,1,86,14,2,0,73.0,GCT,7.943089,0.457711,intronic,2082289,0.928571,617678,0
119920,chrX,638533,24,2,689,30,15,0,54.5,CCC,3.037313,0.542289,intergenic,2061435,0.600000,638532,0
119928,chrX,811178,4,2,103,11,4,0,11.0,TTT,3.709402,0.368159,intergenic,1888790,0.454545,811177,0
